In [1]:
import csv
import numpy as np
import pandas as pd 
from datetime import date
from datetime import datetime

import fastf1 as ff1
from fastf1.core import Laps
from fastf1 import utils
from fastf1 import plotting
import csv
import unicodedata

C:\Users\jornv\Programs\lib\site-packages\fastf1\plotting.py:37: UserWarning: Failed to import optional dependency 'matplotlib'!Plotting functionality will be unavailable!
  warnings.warn("Failed to import optional dependency 'matplotlib'!"
C:\Users\jornv\Programs\lib\site-packages\fastf1\plotting.py:42: UserWarning: Failed to import optional dependency 'timple'!Plotting of timedelta values will be restricted!
  warnings.warn("Failed to import optional dependency 'timple'!"


In [2]:
ff1.Cache.enable_cache(r'C:\Users\jornv\OneDrive\Documenten\Fast f1 Cache')

In [24]:
races = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\races.csv", on_bad_lines='skip', header= 0, delimiter= ',')
drivers = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\drivers.csv", on_bad_lines='skip', header= 0, delimiter= ',')
constructors = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\constructors.csv", on_bad_lines='skip', header= 0, delimiter= ',')

In [19]:
def ff1_retriever(races_df, racetype):
    laps_total = pd.DataFrame()
    messages_total = pd.DataFrame()
    results_total = pd.DataFrame()
    weather_total = pd.DataFrame()
    races_df.replace(r'\N', np.NaN, inplace=True)
    races_df = races_df.astype({"date": "datetime64", "quali_date": "datetime64"})
    
    races_miss = []  # Initialize races_miss list
    
    if racetype == 'Q':
        qualifying = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\qualifying.csv", on_bad_lines='skip', header=0, delimiter=',')
        races_miss = qualifying["raceId"].unique().tolist()  # Update races_miss list with qualifying raceIds
    
    if racetype == 'R':
        results = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\results.csv", on_bad_lines='skip', header=0, delimiter=',')
        races_miss = results["raceId"].unique().tolist()  # Update races_miss list with results raceIds
    
    races_df = races_df[~races_df['raceId'].isin(races_miss)]
    races_df['name_short'] = races_df['name'].str.replace(' Grand Prix', '')

    for index, row in races_df.iterrows():
        race = row['name_short']
        year = row['year']
        raceId = row['raceId']
        date = row['quali_date']
        
        if date <= pd.to_datetime('today'):
            print(f"Retrieving data for year {year} and race {race}")
            try:
                qualification = ff1.get_session(year, f'{race}', racetype)
                qualification.load()

                Q_laps = qualification.laps
                Q_messages = qualification.race_control_messages
                Q_results = qualification.results
                Q_weather = qualification.weather_data

                Q_messages['Year'] = year
                Q_laps["Year"] = year
                Q_results["Year"] = year
                Q_weather["Year"] = year
                
                Q_messages['raceId'] = raceId
                Q_laps['raceId'] = raceId
                Q_results['raceId'] = raceId
                Q_weather['raceId'] = raceId

                laps_total = laps_total.append(Q_laps)
                messages_total = messages_total.append(Q_messages)
                results_total = results_total.append(Q_results)
                weather_total = weather_total.append(Q_weather)
                print("Succeeded in retrieving data")
            except:
                print("Failed to retrieve data")
                pass
            
    return laps_total, messages_total, results_total, weather_total

In [20]:
# Function to remove accents and convert to lowercase
def normalize_text(text):
    # Normalize text to NFD form (Decomposes characters into base characters and combining marks)
    normalized = unicodedata.normalize('NFD', text)
    # Filter out the combining marks (accents)
    no_accents = ''.join([c for c in normalized if unicodedata.category(c) != 'Mn'])
    # Convert to lowercase
    lower_case = no_accents.lower()
    return lower_case

In [21]:
Q_laps, Q_messages, Q_results, Q_weather = ff1_retriever(races, 'Q')
R_laps, R_messages, R_results, R_weather = ff1_retriever(races, 'R')

Retrieving data for year 2024 and race Saudi Arabian


core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v2.2.7]
api            INFO 	No cached data found for driver_info. Loading data...
api            INFO 	Fetching driver list...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_data. Loading data...
api            INFO 	Fetching timing data...
api            INFO 	Parsing timing data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for timing_app_data. Loading data...
api            INFO 	Fetching timing app data...
api            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
api            INFO 	No cached data found for session_status_data. Loading data...
api            INFO 	Fetching session status data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for track_status_data. Loading data...
api            INFO 	Fetchi

Succeeded in retrieving data


## Adding new qualifying data to qualifying.csv

In [25]:
drivers['forename_corrected'] = drivers['forename'].apply(normalize_text)
drivers['surname_corrected'] = drivers['surname'].apply(normalize_text)

Q_results['FirstName_corrected'] = Q_results['FirstName'].apply(normalize_text)
Q_results['LastName_corrected'] = Q_results['LastName'].apply(normalize_text)

qualifying_df = Q_results.merge(drivers, left_on = ['FirstName_corrected', 'LastName_corrected'], right_on = ['forename_corrected', 'surname_corrected'], how = 'left')

print(f"Drivers: {qualifying_df[qualifying_df['forename'].isna()][['FirstName', 'LastName']].drop_duplicates().to_string(index=False, header=False)} are missing")

Drivers: Empty SessionResults
Columns: [FirstName, LastName]
Index: [] are missing


In [26]:
constructors['name_corrected'] = constructors['name'].apply(normalize_text)

qualifying_df['TeamName_corrected'] = qualifying_df['TeamName'].apply(normalize_text)

qualifying_df = qualifying_df.merge(constructors, left_on = ['TeamName_corrected'], right_on = ['name_corrected'], how = 'left')

print(f"Constructors: {qualifying_df[qualifying_df['name_corrected'].isna()][['TeamName_corrected']].drop_duplicates().to_string(index=False, header=False)} are missing")

Constructors: Empty SessionResults
Columns: [TeamName_corrected]
Index: [] are missing


In [27]:
qualifying_df = qualifying_df[['raceId', 'driverId', 'constructorId', 'DriverNumber', 'Position', 'Q1', 'Q2', 'Q3']]
qualifying_df.columns = ['raceId', 'driverId', 'constructorId', 'number', 'position', 'q1', 'q2', 'q3']

In [28]:
Q_df = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\qualifying.csv", on_bad_lines='skip', header=0, delimiter=',')
next_id = Q_df['qualifyId'].max() + 1
qualifying_df.insert(0, 'qualifyId', range(next_id, next_id + len(qualifying_df)))
Q_df = Q_df.append(qualifying_df)
Q_df.to_csv(r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\qualifying.csv", mode='w', header=True, index=False)

## Adding new result data to result.csv

In [8]:
results_df = R_results.merge(drivers, left_on = ['FirstName', 'Abbreviation'], right_on = ['forename', 'code'], how = 'left')
results_df = results_df.merge(constructors, left_on = 'TeamName', right_on = 'name', how = 'left')


In [9]:
R_laps['fastest_time'] = R_laps.groupby(['DriverNumber', 'raceId'])['LapTime'].transform('min')
quickest_lap = R_laps[R_laps['fastest_time'] == R_laps['LapTime']]
results_df = results_df.merge(quickest_lap, on = ['DriverNumber', 'raceId'], how = 'left')
results_df = results_df[['raceId', 'driverId', 'constructorId', 'DriverNumber', 'GridPosition', 'Position', 'Points', 'LapNumber', 'LapTime', 'SpeedFL']]
results_df.columns = ['raceId', 'driverId', 'constructorId', 'number', 'grid', 'positionOrder', 'points', 'fastestLap', 'fastestLapTime', 'fastestLapSpeed']
results_df['position'] = results_df['positionOrder']
results_df['positionText'] = results_df['positionOrder']
results_df = results_df.sort_values(by=['raceId', 'positionOrder'], ascending=[True, True])
results_df['rank'] = results_df.groupby('raceId')['fastestLapTime'].rank(ascending=False)

In [10]:
R_df = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Formula 1 from kaggle\results.csv", on_bad_lines='skip', header= 0, delimiter= ',')
next_id = R_df['resultId'].max() + 1
results_df.insert(0, 'resultId', range(next_id, next_id + len(results_df)))
R_df = R_df.append(results_df)
R_df.to_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Fact\results.csv", mode='w', header=True, index=False)

## Adding new driverstanding data to driverstanding.csv

In [11]:
results_df = results_df.merge(races, on = 'raceId', how = 'left')

In [12]:
results_df['win'] = results_df['positionOrder'].apply(lambda x: 1 if x == 1 else 0)

In [13]:
driverpoint = []
driver_standings_df = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Formula 1 from kaggle\driver_standings.csv", on_bad_lines='skip', header= 0, delimiter= ',')
driver_standings = driver_standings_df["raceId"].unique()
for index, row in results_df.iterrows():
    driverId = row['driverId']
    raceId = row['raceId']
    year = row['year']
    roundId = row["round"]
    if raceId not in driver_standings:
        driver_results = results_df[(results_df['driverId'] == driverId) & (results_df['year'] == year) & (results_df['round'] <= roundId)]['points'].sum()
        win = results_df[(results_df['driverId'] == driverId) & (results_df['year'] == year) & (results_df['raceId'] <= raceId)]['win'].sum()
        driverpoint.append((raceId, driverId, driver_results, win))
        
driverpoint_df = pd.DataFrame(driverpoint, columns =['raceId', 'driverId',  "points", "wins"])
driverpoint_df['position'] = driverpoint_df.groupby('raceId')['points'].rank(ascending=False)
driverpoint_df['positionText'] = driverpoint_df['position']

In [14]:

next_id = driver_standings_df['driverStandingsId'].max() + 1

driverpoint_df.insert(0, 'driverStandingsId', range(next_id, next_id + len(driverpoint_df)))

driver_standings_df = driver_standings_df.append(driverpoint_df)

driver_standings_df.to_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Fact\driver_standings.csv", mode='w', header=True, index=False)

## Adding new constructorstanding data to constructorstanding.csv

In [15]:
results_df = results_df.merge(driverpoint_df, on = ['raceId', 'driverId'], how = "left")

In [16]:
constructors_standings = results_df.groupby(['raceId', 'constructorId'], as_index = False)[['points_y', 'wins']].sum()
constructors_standings = constructors_standings.apply(lambda x: x.reset_index(drop=True))
constructors_standings['position'] = constructors_standings.groupby('raceId')['points_y'].rank(ascending=False)
constructors_standings['positionText'] = constructors_standings['position']

In [17]:
constructors_standings = constructors_standings[['raceId', 'constructorId', 'points_y', 'position', 'positionText', 'wins']]
constructors_standings.columns = ['raceId', 'constructorId', 'points', 'position', 'positionText', 'wins']


In [18]:
constructor_standings_df = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Formula 1 from kaggle\constructor_standings.csv", on_bad_lines='skip', header= 0, delimiter= ',')

next_id = constructor_standings_df['constructorStandingsId'].max() + 1

constructors_standings.insert(0, 'constructorStandingsId', range(next_id, next_id + len(constructors_standings)))

constructor_standings_df = constructor_standings_df.append(constructors_standings)

constructor_standings_df.to_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Fact\constructor_standings.csv", mode='w', header=True, index=False)

## Adding new laps data to lap_times.csv

In [19]:
#results_df = results_df.merge(drivers, on = 'driverId', how = 'left')
laps_df = R_laps.merge(results_df[['driverId', 'number']].drop_duplicates(), left_on = ['DriverNumber'], right_on = ['number'], how = 'left')

In [20]:
laps_df = laps_df[['raceId', 'driverId', 'LapNumber', 'LapTime']]
laps_df = laps_df.sort_values(by=['raceId', 'driverId', 'LapNumber'], ascending=[True, True, True])

In [21]:
laps_df['milliseconds'] = laps_df['LapTime'].dt.total_seconds() * 1000
laps_df['CumulativeSum'] = laps_df.groupby(['driverId', 'raceId'])['milliseconds'].cumsum()
# Convert 'milliseconds' column to seconds
laps_df['seconds'] = laps_df['milliseconds'] / 1000
laps_df['seconds'] = laps_df['seconds'].fillna(0)
# Get minutes, seconds and milliseconds
laps_df['time'] = laps_df['seconds'].apply(lambda x: f"{int(x // 60):02d}:{int(x % 60):02d}.{int((x % 1)*1000):06d}")

# If you don't need the temporary 'seconds' column anymore, you can drop it
laps_df = laps_df.drop(['seconds'], axis=1)


In [22]:
laps_df['position'] = laps_df.groupby(['raceId', 'LapNumber'])['CumulativeSum'].rank(ascending=True)

In [23]:
lap_times_df = pd.read_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Formula 1 from kaggle\lap_times.csv", on_bad_lines='skip', header= 0, delimiter= ',')

In [24]:
laps_df = laps_df[['raceId', 'driverId', 'LapNumber', 'position', 'time', 'milliseconds']]
laps_df.columns = ['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds']

In [25]:
lap_times_df = lap_times_df.append(laps_df)
lap_times_df.to_csv(r"C:\Users\jornv\OneDrive\Documenten\Python data\Fact\lap_times.csv", mode='w', header=True, index=False)

In [26]:
R_laps.info()

<class 'fastf1.core.Laps'>
Int64Index: 4257 entries, 0 to 879
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Time                4257 non-null   timedelta64[ns]
 1   DriverNumber        4257 non-null   object         
 2   LapTime             4059 non-null   timedelta64[ns]
 3   LapNumber           4257 non-null   float64        
 4   Stint               4257 non-null   float64        
 5   PitOutTime          271 non-null    timedelta64[ns]
 6   PitInTime           201 non-null    timedelta64[ns]
 7   Sector1Time         4149 non-null   timedelta64[ns]
 8   Sector2Time         4253 non-null   timedelta64[ns]
 9   Sector3Time         4253 non-null   timedelta64[ns]
 10  Sector1SessionTime  4149 non-null   timedelta64[ns]
 11  Sector2SessionTime  4253 non-null   timedelta64[ns]
 12  Sector3SessionTime  4253 non-null   timedelta64[ns]
 13  SpeedI1             3534 non-null   float64  